In [ ]:
import pandas as pd
import urllib.request
import io

In [ ]:
meta_data = pd.read_csv("../data/raw_shapes/ESP_meta.csv")[["STATION", "NAME", "SQ_KM"]]

In [ ]:
#consistency
meta_data.at[314, "STATION"] = 38001
meta_data.at[315, "STATION"] = 39001

In [ ]:
for idx, area in meta_data[["STATION", "SQ_KM"]].values[::, ::]:
    
    idx = str(int(idx))
    
    print(f"{idx}")
    
    request = f"http://nrfaapps.ceh.ac.uk/nrfa/ws/time-series/{idx}?station={idx}&data-type=gdf&format=nrfa-csv"
    
    runoff = pd.read_csv(io.BytesIO(urllib.request.urlopen(request).read()), 
                         skiprows=21, header=None, usecols=[0, 1], index_col=0, parse_dates=True, names=["date", "Obs"])
      
    coef = ((86400*1000)/(area*1000000))
    
    
    runoff["Obs"] = runoff["Obs"] * coef
    
    runoff = runoff["2008-01-02":"2015-12-31"]
    
    runoff.to_pickle(f"../data/runoff/{idx}.pkl")